# 2. HPC PDE Solver - The diffusion part

## Requirements

In order to fully understand this notebook, please be sure to have already taken a look at the notebooks:
* **0_hpc_mpi_tutorial**
* **1_hcp_pde_data**

**As before it is adviced to run this Jupyter notebook with 4 processes**. 


## The diffusion part of the mini-app

This part will be dedicated for one function only, namely the diffusion function, this function reproduces exactly the stencil operator (see Figure below). As a quick reminder, remember that, after having discretized the Fisher's equation, **each** grid point, in order to update its new $S$ value, must solve the following equation:

$f_{i,j}^{k} := -(4+\alpha)S_{i,j}^{k}+S_{i-1,j}^{k}+S_{i+1,j}^{k}+S_{i,j-1}^{k}+S_{i,j+1}^{k}+\beta S_{i,j}^{k}(1-S_{i,j}^{k}) +\alpha S_{i,j}^{k-1}$

And as this process must be repeated for each grid point, we get a set of non-linear equations to solve. The solving process will come in the next notebook. For now, we only want to get the $f_{i,j}^{k}$ matrix of our grid and it's exactly what the diffusion function will do. 

### The idea

Basically, the idea will be to apply the equation above on each grid point. As we can see, for the current grid point, the equation needs the value of the point on the right, above, on the left and below as well as the value of the previous solution to that point. We can illustrate this process like this (**keep in mind that the same process occurs for every point of the grid**): 

![title](media/picture3.png)

### The code

As always we initialize the Ipyparallel client: 

In [1]:
import ipyparallel as ipp
rc = ipp.Client()
print("There are " + str(len(rc)) + " processes running in parallel")
print("IDs of the processes running in parallel", rc.ids)

There are 4 processes running in parallel
IDs of the processes running in parallel [0, 1, 2, 3]


In [2]:
%%px

from mpi4py import MPI as mpi
import numpy as np
import time

In [3]:
%%px

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

print("Hello from process " + str(rank))

[stdout:0] Hello from process 0
[stdout:1] Hello from process 1
[stdout:2] Hello from process 2
[stdout:3] Hello from process 3


We also need to add our previous code for the **data structures** (you can ignore this cell if you understood the previous notebook):

In [4]:
%%px

class Discretization: 
    
    def __init__(self, nx, ny, nt, t, d=1.0, r=1000.0, v=0, ic=None):
        self.nx = int(nx)                             # x dimension (Number of HORIZONTAL grid points) (int)
        self.ny = int(ny)                             # y dimension (Number of VERTICAL grid points) (int)
        self.nt = int(nt)                             # number of time steps (int)
        self.dx = 1.0 / (nx - 1)                      # distance between each grid points (vertically and horizontally) (double)
        self.dt = t / nt                              # time step size (double)
        self.d = d                                    # the diffusion coefficient D (double)
        self.r = r                                    # the reaction coefficient R (double)
        self.alpha = (self.dx**2)/(self.d * self.dt)  # dx^2/(D*dt) (double)
        self.beta = self.r * (self.dx**2) / self.d    # R*dx^2/D (double)
        
        # simulation parameters
        self.verbose_output = False      # if you want to print more informations during the simulation
        self.custom_init = False         # if the simulation has a customised initial condition
        
        # verbose output
        if v != 0:
            self.verbose_output = True 
        
        # we check that the points entered are valid for the customised initial condition
        # None is equivalent to False when evaluated in Python
        if ic: 
            for point in ic:
                    assert(point[0] >= 0 and point[0] < self.nx)  # checking x coordinate
                    assert(point[1] >= 0 and point[1] < self.ny)  # checking y coordinate
                    assert(point[2] >= 0.0 and point[2] <= 1.0)   # checking s value
            self.points = ic                                        # the custom initial condition is valid
            self.custom_init = True
       
        # statistics variables initialized
        self.flops_count = 0   # number of flops
        self.iters_cg = 0      # number of Conjugate Gradient iteration
        self.iters_newton = 0  # number of Newton iteration

class Domain: 

    def create_dim(self, size):
        dividers = []
        for i in range(size - 1 , 0, -1):
            if (size%i == 0 and i != 1):
                dividers.append(i)
        if (dividers == []):
            return [size, 1]
        else: 
            divider = dividers[len(dividers) // 2]
            return [size // divider, divider]
    
    # constructor for the domain object
    def __init__(self, rank, size, discretization, communicator):

        # dimensions of the cartesian plan
        dims = self.create_dim(size)
        
        # save the dimension on each axis as an object attribute
        self.ndomy = dims[0]
        self.ndomx = dims[1]

        # generate the cartesian plan given the dimensions (mpi function)
        # no periodicity of the dimensions
        # no reordering
        comm_cart = communicator.Create_cart(dims, [False, False], False)
        
        # save the cartesian communication group as an object attribute
        self.comm_cart = comm_cart

        # get the coordinates of the current processor on the newly generated cartesian plan (mpi function)
        coords = self.comm_cart.Get_coords(rank)
        
        # save the coordinates as attributes
        self.domy = coords[0] # (int)
        self.domx = coords[1] # (int)
        
        # get the ranks of the domains neighbouring the current domain
        # .Shift(direction_on_cartesian_plan, size_of_the_shift)
        # row direction is 0 (x-direction)
        # column direction is 1 (y-direction)
        south_north = self.comm_cart.Shift(0, 1) # tuple (int, int)
        west_east = self.comm_cart.Shift(1, 1) # tuple (int, int)
    
        # save the rank of neighbouring domains as object attributes
        self.neighbour_south = south_north[0]
        self.neighbour_north = south_north[1]
        self.neighbour_west = west_east[0]
        self.neighbour_east = west_east[1]

        # number of horizontal and vertical grid points of the current domain
        self.nx = discretization.nx // self.ndomx # (int)
        self.ny = discretization.ny // self.ndomy # (int)

        # the starting coordinates of the current (sub)domain in the full grid
        # the coordinates start from 0
        self.startx = (self.domx) * self.nx # (int)
        self.starty = (self.domy) * self.ny # (int)

        # we adjust for grid dimensions that, potentially, do not divide evenly between the domains
        # we stretch the last element of each dimension to fit the remaining part
        # domx and domy start from 0 so we have to do (ndomx - 1) & (ndomy - 1)
        if self.domx == (self.ndomx - 1):
            self.nx = discretization.nx - self.startx
        if self.domy == (self.ndomy - 1):
            self.ny = discretization.ny - self.starty
        
        # the ending coordinates in the full grid of the current domain
        self.endx = self.startx + self.nx - 1 # (int)
        self.endy = self.starty + self.ny - 1 # (int)

        # the total number of grid points of the current domain
        self.n_total = self.nx * self.ny # (int)

        # mpi values for the current domain saved as object attributes
        self.rank = rank
        self.size = size
        
        # fields holding the solutions (Numpy matrices)
        self.x_new = np.zeros((self.ny, self.nx), dtype=np.float64) # solution at timestep k (2d)
        self.x_old = np.zeros((self.ny, self.nx), dtype=np.float64) # solution at timestep k-1 (2d)
       
        # fields holding the boundary points when they are received from the neighbouring domains
        self.bndN = np.zeros((1, self.nx), dtype=np.float64) # boundary north (1d)
        self.bndS = np.zeros((1, self.nx), dtype=np.float64) # boundary east (1d)
        self.bndE = np.zeros((1, self.ny), dtype=np.float64) # boundary south (1d)
        self.bndW = np.zeros((1, self.ny), dtype=np.float64) # boundary east (1d)

        # buffers used during boundaries communication
        # to send the boundary points of the current domain
        # to its neighbours
        self.buffN = np.zeros((1, self.nx), dtype=np.float64) # (1d)
        self.buffS = np.zeros((1, self.nx), dtype=np.float64) # (1d)
        self.buffE = np.zeros((1, self.ny), dtype=np.float64) # (1d)
        self.buffW = np.zeros((1, self.ny), dtype=np.float64) # (1d)
     
    # function for printing the parameters of the cartesian division among each process
    def print(self):
        # print, for each process, the domain object
        for i in range(0, self.size):
            if i == self.rank:  
                print("Rank " + str(self.rank) + "/" + str(self.size-1))
                print("At index (" + str(self.domy) + "," + str(self.domx) + ")")
                print("Neigh N:S " + str(self.neighbour_north) + ":" + str(self.neighbour_south))
                print("Neigh E:W " + str(self.neighbour_east) + ":" + str(self.neighbour_west))
                print("Coordinates startx:endx  " + str(self.startx) + ":" + str(self.endx))
                print("Coordinates starty:endy  " + str(self.starty) + ":" + str(self.endy))
                print("Local dims " + str(self.nx) + " x " + str(self.ny))
                print("")
            
            # keep the printing ordered and cleaned
            MPI.COMM_WORLD.Barrier()
            # wait a bit when printing to avoid polluating other printed messages.
            time.sleep(0.1)

        return None

All the computations in the diffusion function are made in matricial form using Numpy, not only this form is highly efficient, but it also makes the equations simpler to read by avoiding multiple loops. The only thing to keep in mind when doing this calculation with Numpy is the fact that we use a **slice operators**. A slice operator is simply a range of value, let's look at an example:

In [5]:
%%px

a = np.array([[1, 2, 3, 4, 5], 
              [6, 7, 8, 9, 10], 
              [11, 12, 13, 14, 15], 
              [16, 17, 18, 19, 20], 
              [21, 22, 23, 24, 25]]) # a 5x5 Numpy matrix

if rank==0:
    # we want to print everything of matrix a but not the last column
    # the slice on the left of the coma is for rows
    # the slice on the right of the coma is for columns, here, column 0 is included but 4 is not included
    print("We want everything but the last column of a:\n", a[:, 0:4])
    # Another example for the interior grid points of a (assuming a is a grid)
    print("Interior grid points of a:\n", a[1:4, 1:4])

[stdout:0] 
We want everything but the last column of a:
 [[ 1  2  3  4]
 [ 6  7  8  9]
 [11 12 13 14]
 [16 17 18 19]
 [21 22 23 24]]
Interior grid points of a:
 [[ 7  8  9]
 [12 13 14]
 [17 18 19]]


### Understanding the diffusion function

For implementing the diffusion function, we will heavily use MPI because, as you have probably already understood, we need to send the boundary points between the domains as illustrated in the figure above where the domain 0, to compute its East boundary, needs the West boundary of the domain 1.

The logic of our function will be as follows: 
* We will use **a non-blocking communication** to send the boundaries of the current domain and receive those of its neighbours.
* While the current process wait for those non-blocking communications to succeed, **the process will calculate the points of its interior grid. Indeed, those points do not need the boundaries to be calculated** (see the example above for the interior grid points of domain 2 in a grey square). We play on the fact that the communications are non-blocking and that they do not stop the code execution to make the process more efficient.
* Once the interior grid points are calculated **and** the non-blocking communications achieved, our current process has the boudaries of its neighbouring domains, so it can now calculate all its boundaries.

So the first step of our function which is also the hardest one is the non-blocking communication of the boundaries of each domain with their neighbours. Let's illustrate a first prototype of diffusion function called `diffusion_test` which only implements a simple boudary communication between North and South neighbours. East and West neighbours are ignored. **In other terms, each domain will have to receive the boundaries from its potential North and South neighbours (if they exist) and send them its own boundaries.** 

The real `diffusion` function and our `diffusion_test` takes the following parameters: 
* U is the current solution `x_new`.
* S is the resulting $f_{i,j}^{k}$ for the full grid after having applied the stencil operator to all points.
* discretization is the discretization object. 
* domain is the domain object. 

In [6]:
%%px

# a very simple prototype of diffusion function implementing only the North-South non-blocking communication to show the functioning
def diffusion_test(U, S, discretization, domain):
        
    # we create shortcuts for variables heavily used 
    nx = domain.nx
    ny = domain.ny
    comm_cart = domain.comm_cart

    # we initialize the lists for saving the mpi requests and statuses
    # of the non-blocking functions in order to wait for them later
    statuses = [MPI.Status()] * 2
    requests = [MPI.Request()] * 2
    num_requests = 0 # current requests counter
    # we initialize the lists for saving the mpi requests and statuses
    # of the non-blocking functions in order to wait for them later
    statuses = [MPI.Status()] * 2
    requests = [MPI.Request()] * 2
    num_requests = 0 # current requests counter
    
    # send the North boundary to the North neighbour if the current domain has a North neighbour
    if domain.neighbour_north >= 0:
        # wait for the south boundary of the Norht neighbour
        # set tag to be the sender's rank
        # .Irecv([np_array_received, type_of_members], rank_of_source, message_tag)
        # the request is returned and saved to wait for it after
        requests[num_requests] = comm_cart.Irecv([domain.bndN, MPI.DOUBLE], domain.neighbour_north, domain.neighbour_north)
        num_requests += 1
        
        # print immediatly the North boundary (which has not been received yet)
        print("Domain " + str(domain.rank) + " will receive from its North neighbour " + str(domain.neighbour_north) + ":\n", domain.bndN)

        # pack North buffer of the current domain to send it
        # the current domain makes use of the buffer object to send the array
        domain.buffN[0, :] = U[ny-1, :]
        
        # print the boundary which will be sent to North neighbour
        print("Domain " + str(domain.rank) + " will send to its North neighbour " + str(domain.neighbour_north) + ":\n", domain.buffN)
        
        # the current domain sends its North boundary which will be received as the South boundary by its North neighbour
        # .Isend([np_array_sent, type_of_members], rank_of_destination, message_tag)
        requests[num_requests] = comm_cart.Isend([domain.buffN, MPI.DOUBLE], domain.neighbour_north, domain.rank)
        num_requests += 1
        
    # same logic for the South boundary, send it to the South neighbour if the current domain has a South neighbour
    if domain.neighbour_south >= 0:
        
        # receive the North boundary of the South neighbour
        requests[num_requests] = comm_cart.Irecv([domain.bndS, MPI.DOUBLE], domain.neighbour_south, domain.neighbour_south)
        
        # ----------------------------------------------------
        # !! we wait for the boundary of South neighbour !!
        # be careful to not put this wait function also after the .Irecv of the North domain otherwise you will 
        # enter a deadlock between processes
        # here we wait on purpose to see the result of the exchange
        requests[num_requests].Wait()
        # ----------------------------------------------------
        
        num_requests += 1
        
        print("Domain " + str(domain.rank) + " will receive from its South neighbour " + str(domain.neighbour_south) + ":\n", domain.bndS)
        
        # pack the South boundary into the South buffer to send it to the South neighbour
        domain.buffS[0, :] = U[0, :]
        
        print("Domain " + str(domain.rank) + " will send to its South neighbour " + str(domain.neighbour_south) + ":\n", domain.buffS)
        
        # send the South boundary to the South neighbour
        requests[num_requests] = comm_cart.Isend([domain.buffS, MPI.DOUBLE], domain.neighbour_south, domain.rank)
        num_requests += 1

We thus now initialize the domain 0 matrix `x_new` with some values for its South boundary in order to see the results. Indeed, domain 2 should receive the South boundary of domain 0 as its North boundary. 

In [7]:
%%px

# we initialize the objects
discretization = Discretization(4, 4, 100, 0.01)
domain = Domain(rank, 4, discretization, comm)

# let's put some values in the South boudary of domain 0
if rank == 0:
    domain.x_new[1, 0] = 0.1
    domain.x_new[1, 1] = 0.2 
    print("x_new for domain 0:\n", domain.x_new, "\n")

S = np.zeros((domain.ny, domain.nx),  dtype=np.float64)
# we use the diffusion_test function to see the exchange North-South
diffusion_test(domain.x_new, S, discretization, domain)

[stdout:0] 
x_new for domain 0:
 [[0.  0. ]
 [0.1 0.2]] 

Domain 0 will receive from its North neighbour 2:
 [[0. 0.]]
Domain 0 will send to its North neighbour 2:
 [[0.1 0.2]]
[stdout:1] 
Domain 1 will receive from its North neighbour 3:
 [[0. 0.]]
Domain 1 will send to its North neighbour 3:
 [[0. 0.]]
[stdout:2] 
Domain 2 will receive from its South neighbour 0:
 [[0.1 0.2]]
Domain 2 will send to its South neighbour 0:
 [[0. 0.]]
[stdout:3] 
Domain 3 will receive from its South neighbour 1:
 [[0. 0.]]
Domain 3 will send to its South neighbour 1:
 [[0. 0.]]


As you can see, the exchange between domain 0 and its North neighbour, domain 2, works. 

Now, let's do the same test but,  **this time with the `.Waitall()` function**. We will remove the `.Wait()` in South communication and just put the `.Waitall()` at the end of all the communications to see what is happening. 

In [8]:
%%px

# a very simple prototype of diffusion function implementing only the North-South non-blocking communication to show the functioning
def diffusion_test(U, S, discretization, domain):
        
    # we create shortcuts for variables heavily used 
    nx = domain.nx
    ny = domain.ny
    comm_cart = domain.comm_cart

    # we initialize the lists for saving the mpi requests and statuses
    # of the non-blocking functions in order to wait for them later
    statuses = [MPI.Status()] * 2
    requests = [MPI.Request()] * 2
    num_requests = 0 # current requests counter
    
    # send the North boundary to the North neighbour if the current domain has a North neighbour
    if domain.neighbour_north >= 0:
        # wait for the south boundary of the Norht neighbour
        # set tag to be the sender's rank
        # .Irecv([np_array_received, type_of_members], rank_of_source, message_tag)
        # the request is returned and saved to wait for it after
        requests[num_requests] = comm_cart.Irecv([domain.bndN, MPI.DOUBLE], domain.neighbour_north, domain.neighbour_north)
        num_requests += 1
        
        # print immediatly the North boundary (which has not been received yet)
        print("Domain " + str(domain.rank) + " will receive from its North neighbour " + str(domain.neighbour_north) + ":\n", domain.bndN)

        # pack North buffer of the current domain to send it
        # the current domain makes use of the buffer object to send the array
        domain.buffN[0, :] = U[ny-1, :]
        
        # print the boundary which will be sent to North neighbour
        print("Domain " + str(domain.rank) + " will send to its North neighbour " + str(domain.neighbour_north) + ":\n", domain.buffN)
        
        # the current domain sends its North boundary which will be received as the South boundary by its North neighbour
        # .Isend([np_array_sent, type_of_members], rank_of_destination, message_tag)
        requests[num_requests] = comm_cart.Isend([domain.buffN, MPI.DOUBLE], domain.neighbour_north, domain.rank)
        num_requests += 1
        
    # same logic for the South boundary, send it to the South neighbour if the current domain has a South neighbour
    if domain.neighbour_south >= 0:
        
        # receive the North boundary of the South neighbour
        requests[num_requests] = comm_cart.Irecv([domain.bndS, MPI.DOUBLE], domain.neighbour_south, domain.neighbour_south)
        num_requests += 1
        
        print("Domain " + str(domain.rank) + " will receive from its South neighbour " + str(domain.neighbour_south) + ":\n", domain.bndS)
        
        # pack the South boundary into the South buffer to send it to the South neighbour
        domain.buffS[0, :] = U[0, :]
        
        print("Domain " + str(domain.rank) + " will send to its South neighbour " + str(domain.neighbour_south) + ":\n", domain.buffS)
        
        # send the South boundary to the South neighbour
        requests[num_requests] = comm_cart.Isend([domain.buffS, MPI.DOUBLE], domain.neighbour_south, domain.rank)
        num_requests += 1
    
    # ----------------------------------------------------
    # !! waiting for all non-blocking communications made to succeed
    MPI.Request.Waitall(requests, statuses)
    # ----------------------------------------------------
    
    print("---------------------------------------------------------------------")                                             
    print("After the Waitall function")
    print("Domain " + str(domain.rank) + " has South boundary:\n", domain.bndS)
    print("Domain " + str(domain.rank) + " has North boundary:\n", domain.bndN) 
    print("")

We use the same objects as before. Because we removed the `.Wait()`, you should see that the boundary sent by domain 0 is now $[0, 0]$ meaning that we print before getting the actual boundary (**as it is a non-blocking communication**). But now, after the `.Waitall` function, we get the domains with all the boundaries received:

In [9]:
%%px

discretization = Discretization(4, 4, 100, 0.01)
domain = Domain(rank, 4, discretization, comm)

if rank == 0:
    domain.x_new[1, 0] = 0.1
    domain.x_new[1, 1] = 0.2 
    print("x_new for domain 0:\n", domain.x_new, "\n")

S = np.zeros((domain.ny, domain.nx),  dtype=np.float64)
diffusion_test(domain.x_new, S, discretization, domain)

[stdout:0] 
x_new for domain 0:
 [[0.  0. ]
 [0.1 0.2]] 

Domain 0 will receive from its North neighbour 2:
 [[0. 0.]]
Domain 0 will send to its North neighbour 2:
 [[0.1 0.2]]
---------------------------------------------------------------------
After the Waitall function
Domain 0 has South boundary:
 [[0. 0.]]
Domain 0 has North boundary:
 [[0. 0.]]

[stdout:1] 
Domain 1 will receive from its North neighbour 3:
 [[0. 0.]]
Domain 1 will send to its North neighbour 3:
 [[0. 0.]]
---------------------------------------------------------------------
After the Waitall function
Domain 1 has South boundary:
 [[0. 0.]]
Domain 1 has North boundary:
 [[0. 0.]]

[stdout:2] 
Domain 2 will receive from its South neighbour 0:
 [[0. 0.]]
Domain 2 will send to its South neighbour 0:
 [[0. 0.]]
---------------------------------------------------------------------
After the Waitall function
Domain 2 has South boundary:
 [[0.1 0.2]]
Domain 2 has North boundary:
 [[0. 0.]]

[stdout:3] 
Domain 3 will rec

Again, only look at the exchange between domain 0 and 2. Now that non-blocking communication is clear. We can do the real diffusion function by adding: 
* The West-East communication. 
* The interior grid points computations after the non-blocking communications but before the `.Waitall` function. 
* The boundaries points computations after the `.Waitall` function.

**This complete diffusion function has been exhaustively commented.**

In [10]:
%%px

# the complete diffusion function
def diffusion(U, S, discretization, domain):
        
    # we create shortcuts for variables heavily used in the function
    alpha = discretization.alpha
    beta = discretization.beta
    nx = domain.nx
    ny = domain.ny
    comm_cart = domain.comm_cart

    # we initialize the lists for saving the mpi requests and statuses of the non-blocking functions in order to wait for them later
    # there should be a maximum of 8 requests if the domain is completely surrounded by neighbours
    statuses = [MPI.Status()] * 8
    requests = [MPI.Request()] * 8
    num_requests = 0 # current requests counter

    # !! The non-blocking communication !!

    # send the North boundary to the North neighbour if the current domain has a North neighbour
    if domain.neighbour_north >= 0:
        # wait for the south boundary of the Norht neighbour
        # set tag to be the sender's rank
        # .Irecv([np_array_received, type_of_members], rank_of_source, message_tag)
        # the request is returned and saved to wait for it after
        requests[num_requests] = comm_cart.Irecv([domain.bndN, MPI.DOUBLE], domain.neighbour_north, domain.neighbour_north)
        num_requests += 1

        # pack North buffer of the current domain to send it
        # the current domain makes use of the buffer object to send the array
        domain.buffN[0, :] = U[ny-1, :]
        
        # the current domain sends its North boundary which will be received as the South boundary by its North neighbour
        # .Isend([np_array_sent, type_of_members], rank_of_destination, message_tag)
        requests[num_requests] = comm_cart.Isend([domain.buffN, MPI.DOUBLE], domain.neighbour_north, domain.rank)
        num_requests += 1
    
    # same logic for South boundary if there is a South neighbour
    if domain.neighbour_south >= 0:

        requests[num_requests] = comm_cart.Irecv([domain.bndS, MPI.DOUBLE], domain.neighbour_south, domain.neighbour_south)
        num_requests += 1

        domain.buffS[0, :] = U[0, :]

        requests[num_requests] = comm_cart.Isend([domain.buffS, MPI.DOUBLE], domain.neighbour_south, domain.rank)  
        num_requests += 1

    # same logic for East boundary if there is a East neighbour
    if domain.neighbour_east >= 0:

        requests[num_requests] = comm_cart.Irecv([domain.bndE, MPI.DOUBLE], domain.neighbour_east, domain.neighbour_east)
        num_requests += 1
        
        domain.buffE[0, :] = U[:, nx-1]
    
        requests[num_requests] = comm_cart.Isend([domain.buffE, MPI.DOUBLE], domain.neighbour_east, domain.rank)  
        num_requests += 1

    # same logic for West boundary if there is a West neighbour
    if domain.neighbour_west >= 0:
        
        requests[num_requests] = comm_cart.Irecv([domain.bndW, MPI.DOUBLE], domain.neighbour_west, domain.neighbour_west)
        num_requests += 1
        
        domain.buffW[0, :] = U[:, 0]

        requests[num_requests] = comm_cart.Isend([domain.buffW, MPI.DOUBLE], domain.neighbour_west, domain.rank) 
        num_requests += 1
    
    # ----------------------------------------------------------------------
    # while waiting for the non-blocking communications to proceed
    # we can calculate the interior grid points
    # it makes the process more efficient by reducing the overhead
    # of waiting for the communications to proceed (if it was blocking communications)
    # ----------------------------------------------------------------------
    
    # we initialize the value for the slice operator to avoid repeating calculation
    srow = domain.ny - 1
    scol = domain.nx - 1
    
    # interior grid points (refer to the formula of the function f_{i, j}^{k})
    S[1:srow, 1:scol] = ( -(4.0 + alpha) * U[1:srow, 1:scol] 
                        + U[1-1:srow-1, 1:scol] + U[1+1:srow+1, 1:scol] 
                        + U[1:srow, 1-1:scol-1] + U[1:srow, 1+1:scol+1]
                        + beta * U[1:srow, 1:scol] * (1.0 - U[1:srow, 1:scol]) 
                        + alpha * domain.x_old[1:srow, 1:scol] )

    # waiting for all the non-blocking communications to succeed before calculating the boundaries
    # .Waitall takes a list of requests to wait for and a list of statuses to update when the requests succeed
    MPI.Request.Waitall(requests, statuses)

    # East boundary
    # ---------------
    
    # slice operator initialized
    srow = domain.ny - 1
    scol = domain.nx - 1
    
    # East boudary calculation
    S[1:srow, scol] = ( -(4.0 + alpha) * U[1:srow, scol]
                        + U[1-1:srow-1, scol] + U[1+1:srow+1, scol] 
                        + U[1:srow, scol-1] + domain.bndE[0, 1:srow]
                        + beta * U[1:srow, scol] * (1.0 - U[1:srow, scol])
                        + alpha * domain.x_old[1:srow, scol]  )
    
    # West boundary
    # ---------------
    
    # slice operator initialized
    srow = domain.ny - 1
    scol = 0
    
    # West boundary calculation
    S[1:srow, scol] = ( -(4.0 + alpha) * U[1:srow, scol]
                        + U[1:srow, scol+1] + U[1-1:srow-1, scol] + U[1+1:srow+1, scol]
                        + alpha * domain.x_old[1:srow, scol] + domain.bndW[0, 1:srow]
                        + beta * U[1:srow, scol] * (1.0 - U[1:srow, scol]) )
    
    # North boundary
    # ---------------
    
    # slice operator
    srow = domain.ny - 1
    
    # North-West corner (which needs the value of the two neighbouring domains on the North and on the West)
    scol = 0
    S[srow, scol] = ( -(4.0 + alpha) * U[srow, scol]
                        + U[srow-1, scol] + domain.bndN[0][scol] 
                        + domain.bndW[0, srow] + U[srow, scol+1]
                        + beta * U[srow, scol] * (1.0 - U[srow, scol])
                        + alpha * domain.x_old[srow, scol])
    
    # North boundary
    scol = domain.nx - 1
    S[srow, 1:scol] = ( -(4.0 + alpha) * U[srow, 1:scol]
                        + U[srow, 1-1:scol-1] + U[srow, 1+1:scol+1] + U[srow-1, 1:scol]
                        + alpha * domain.x_old[srow, 1:scol] + domain.bndN[0, 1:scol]
                        + beta * U[srow, 1:scol] * (1.0 - U[srow, 1:scol]) )
    
    # North-East corner
    scol = domain.nx - 1
    S[srow, scol] = ( -(4.0 + alpha) * U[srow, scol]
                        + U[srow, scol-1] + U[srow-1, scol]
                        + alpha * domain.x_old[srow, scol] + domain.bndE[0, srow] + domain.bndN[0, scol]
                        + beta * U[srow, scol] * (1.0 - U[srow, scol]) )

    # South boundary
    # ---------------
    
    # slice operator
    srow = 0
    
    # South-West corner
    scol = 0
    S[srow, scol] = ( -(4.0 + alpha) * U[srow, scol]
                        + U[srow, scol+1] + U[srow+1, scol]
                        + alpha * domain.x_old[srow, scol] + domain.bndW[0, srow] + domain.bndS[0, scol]
                        + beta * U[srow, scol] * (1.0 - U[srow, scol]) )
    
    # South boundary
    scol = domain.nx - 1
    S[srow, 1:scol] = ( -(4.0 + alpha) * U[srow, 1:scol]
                        + U[srow, 1-1:scol-1] + U[srow, 1+1:scol+1] + U[srow+1, 1:scol]
                        + alpha * domain.x_old[srow, 1:scol] + domain.bndS[0, 1:scol]
                        + beta * U[srow, 1:scol] * (1.0 - U[srow, 1:scol]) )

    # South-East corner
    scol = domain.nx - 1
    S[srow, scol] = ( -(4.0 + alpha) * U[srow, scol]
                        + U[srow, scol-1] + U[srow+1, scol]
                        + alpha * domain.x_old[srow, scol] + domain.bndE[0, srow] + domain.bndS[0, scol]
                        + beta * U[srow, scol] * (1.0 - U[srow, scol]) )

    # Statistics (for printing at the end)
    # flop counts
    # 8 flops per point
    discretization.flops_count += (
            + 12 * (nx - 2) * (ny - 2)  # interior points
            + 11 * (nx - 2  +  ny - 2)  # boundaries points
            + 11 * 4 )                  # corner points

It's time to test it, let's initialize a grid of 4x4, this grid will have one point initialised to 0.2 in the boundary of the domain 0:

In [11]:
%%px 

discretization = Discretization(4, 4, 100, 0.01)
domain = Domain(rank, 4, discretization, comm)

if rank == 0:
    domain.x_new[1, 1] = 0.2 
    print("x_new for domain 0:\n", domain.x_new)

S = np.zeros((domain.ny, domain.nx),  dtype=np.float64)
diffusion(domain.x_new, S, discretization, domain)
print("S matrix for domain " + str(rank) + ":\n", S)

[stdout:0] 
x_new for domain 0:
 [[0.  0. ]
 [0.  0.2]]
S matrix for domain 0:
 [[ 0.00000000e+00  2.00000000e-01]
 [ 2.00000000e-01 -2.05244444e+02]]
[stdout:1] 
S matrix for domain 1:
 [[0.  0. ]
 [0.2 0. ]]
[stdout:2] 
S matrix for domain 2:
 [[0.  0.2]
 [0.  0. ]]
[stdout:3] 
S matrix for domain 3:
 [[0. 0.]
 [0. 0.]]


For 4 processes, the previous diffusion went like that: 

Before the diffusion ($U$ matrix):
$$\begin{bmatrix} 0 & 0 & 0 & 0 \\ 0 & 0.2 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \end{bmatrix}$$

After the diffusion ($S$ matrix):
$$\begin{bmatrix} 0 & 0.2 & 0 & 0 \\ 0.2 & -205.244444 & 0.2 & 0 \\ 0 & 0.2 & 0 & 0 \\ 0 & 0 & 0 & 0 \end{bmatrix}$$

We can see that it actually works ! The initial value $0.2$ is diffused !

Everything should be clear by now concerning the diffusion, don't hesitate to modify the code to print more informations if you want to do some tests but, **be careful to the deadlocks** ! As you can see, non-blocking communication is quite difficult to debug and may create situations where it is very uneasy to see who is doing what and if there is a bug in the logic. **That's why non-blocking communication is actually not widely used in practice and programmers prefer blocking communication for its simplicity**.